In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-617eb153-1f9a-7c98-7483-f0b101475265)


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"
df = pd.read_csv(f"{proj_dir}data_clean/clean.csv", low_memory=False).to_numpy()

In [11]:
generator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(98, activation="sigmoid")
])

In [12]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid") # not correct output dim
])

In [13]:
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [14]:
codings_size = 98
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(df).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [15]:
# Although this runs it still has bugs
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1} of {n_epochs}")
        for X_batch in dataset:
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=(100,98))
            generated_data = generator(noise)
            X_batch = X_batch.reshape((1,98))
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)

            y1 = tf.constant([[0.]] * noise.shape[0] + [[1.]] * X_batch.shape[0])
            y1 = np.reshape(y1, (101, 1))

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)

In [16]:
train_gan(gan, df, batch_size, codings_size, n_epochs=1)

Epoch 1 of 1


In [27]:
import os
import re
gans_saved = os.listdir(f"{proj_dir}saved_models/")
model_number = [int(re.sub("[^\d\.]", "", x)) for x in gans_saved]
last_model = max(model_number)

In [28]:
generator.save(f"{proj_dir}saved_models/gan{last_model+1}")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ece884_project/saved_models/gan2/assets


In [19]:
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_data = generator(noise)
generated_data

<tf.Tensor: shape=(32, 98), dtype=float32, numpy=
array([[6.4402295e-15, 1.1054349e-15, 2.3652556e-14, ..., 1.1549994e-12,
        2.1165701e-13, 4.4940546e-13],
       [7.5639728e-13, 1.6471504e-14, 4.5543891e-13, ..., 5.1311726e-11,
        3.1174027e-13, 7.7778409e-12],
       [1.5467072e-12, 2.6745240e-13, 1.2852231e-12, ..., 5.2949797e-12,
        4.0344208e-12, 3.0778154e-11],
       ...,
       [4.0439908e-11, 1.0695271e-12, 1.3542032e-11, ..., 1.0369925e-09,
        4.2855974e-10, 2.3793585e-11],
       [3.3987555e-12, 2.3454047e-13, 1.3387474e-12, ..., 1.6951351e-13,
        1.0441278e-12, 5.2801009e-13],
       [9.9919361e-12, 4.7423350e-12, 3.9190231e-12, ..., 1.5752689e-11,
        3.9119506e-12, 1.2340419e-11]], dtype=float32)>